Crypto Trading Strategy Optimization </br>

In [32]:
#!pip install yfinance --upgrade
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xzvf ta-lib-0.4.0-src.tar.gz
#%cd ta-lib
#!./configure --prefix=/usr
#!make
#!make install
#!pip install Ta-Lib
#!pip install plotly==5.11.0

In [2]:
import os
import sys
import copy
from datetime import datetime, timedelta
import time
import itertools

import pandas as pd
import pandas_datareader as pdr
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import yfinance as yf
import talib as ta
from talib import MA_Type

**1) Defining Data Folder Structures & Parameters**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#data folder structures
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
backtest_output_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/backtest_output'

In [5]:
# pair inverse trade stocks
ticker_list = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'SOL-USD']

# define date range
start_date = '2021-12-24'
end_date = '2023-01-01'
interval = '1h'

# initial capital in USD
initial_capital = 5000

# position sizing
pos_size = round(initial_capital/len(ticker_list),2)

**2) Getting OHLC Data from Yahoo Finance** </b>


*   Get stock data based on ticker list from Yahoo Finance
*   Create each stock data as different dataframe
*   Add Technical Analysis indicators from TA-Lib
*   Pointing working directory to data folder, export dach dataframe as independent csv file 





In [6]:
# changing working directory to data folder's path
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
os.chdir(data_folder)

crypto_list = []

# getting OHLC data from yfinance package, if auto_adjust=True, OHLC data will not have adj close column, use progress=False to get rid of comments
for ticker in ticker_list:
  
  try:
    crypto = ticker.replace('-', '')
    crypto_list.append(crypto)

    globals()[crypto] = pd.DataFrame
    globals()[crypto] = yf.download(ticker, start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
    globals()[crypto] = globals()[crypto].drop('Volume', axis=1)

    # Adding technical indicators
    globals()[crypto]['200EMA'] = ta.EMA(globals()[crypto]['Close'], timeperiod=200) 
    globals()[crypto]['ATR'] = ta.ATR(globals()[crypto]['High'], globals()[crypto]['Low'], globals()[crypto]['Close'], timeperiod=14)
    globals()[crypto]['RSI_14'] = ta.RSI(globals()[crypto]['Close'], timeperiod=14)
    globals()[crypto]['MACD'], globals()[crypto]['MACD_signal'], globals()[crypto]['MACD_hist'] = ta.MACD(globals()[crypto]['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    globals()[crypto]['ADX'] = ta.ADX(globals()[crypto]['High'], globals()[crypto]['Open'], globals()[crypto]['Close'], 14)
    globals()[crypto]['WILL%R'] = ta.WILLR(globals()[crypto]['High'], globals()[crypto]['Low'], globals()[crypto]['Close'], timeperiod=14)
    globals()[crypto]['BBand_U'], globals()[crypto]['BBand_M'], globals()[crypto]['BBand_L'] = ta.BBANDS(globals()[crypto]['Close']) # if add exponential smoothing, matype=MA_Type.T3

    # add/drop columns and clean up
    globals()[crypto] = globals()[crypto].dropna()
    globals()[crypto] = globals()[crypto].reset_index()
    globals()[crypto] = globals()[crypto].rename(columns={'Datetime': 'Date'})
    globals()[crypto][['Position', 'Win Count']] = 0
    globals()[crypto][['Lot Size', 'Equity Value', 'MDD_dollar', 'PNL', 'Holding Period']] = np.NAN
    
    #writing each ticker dataframe into separate .csv files, save to google drive
    #globals()[crypto].to_csv(ticker+'.csv', index=False, header=True)
  except:
    print(globals()[crypto])

**Other Optional Indicators:** </b>

**MACD_hist** - globals()[ticker]['MACD'], globals()[ticker]['MACD_signal'], globals()[ticker]['MACD_hist'] = ta.MACD(globals()[ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9) </b>

**ADX** - globals()[ticker]['ADX'] = ta.ADX(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14) </b>


**ADXR** - globals()[ticker]['ADXR'] = ta.ADXR(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14) </b>

**50 Day Exponential Moving Average** - globals()[ticker]['50EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=50) </b>

**Bollinger Bands** - globals()[ticker]['BBand_U'], globals()[ticker]['BBand_M'], globals()[ticker]['BBand_L'] = ta.BBANDS(globals()[ticker]['Close']) # if add exponential smoothing, matype=MA_Type.T3

**Stochastic Indicators** - globals()[ticker]['Stoch_slowk'], globals()[ticker]['Stoch_slowd'] = ta.STOCH(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'],                                                              fastk_period=21, slowk_period=5, slowk_matype=0, slowd_period=5, slowd_matype=0)    

**3) Building the DataFrame List, Perform Backtesting**

In [7]:
print(crypto_list)

['BTCUSD', 'ETHUSD', 'BNBUSD', 'SOLUSD']


In [8]:
# list of dataframes to be iterated after yfinance data pull 
df_list = [BTCUSD, ETHUSD, BNBUSD, SOLUSD]

In [9]:
# function to extract dataframe name
def get_df_name(df):
   name =[x for x in globals() if globals()[x] is df][0]
   return name

In [10]:
# function to print function name as a string
def get_func_name(func_name):
    func_name = sys._getframe().f_code.co_name
    return func_name

# **Testing out Various Strategies**

**1) MACD Bollinger Bands Strategy**

*   Entry Rule: Open Price < Lower Bollinger Band, MACD > MACD signal line > 0
*   Exit Rule: Hit profit target/ stop loss/ last day of trade, MACD < signal line



In [11]:
# defining the global list for storing trade list output
global bb_macd_tradelist
bb_macd_tradelist = []

pos_size = round(initial_capital/len(ticker_list),2)

In [12]:
# defining a backtest function based on trading rules, capture all transactions

def backtest_bbmacd(df):
  df_name = get_df_name(df)

  pos_opened = False
  open_price  = 0
  close_price = 0
  lot_size = 0

  initial_equity_value = 0
  win_counter = 0
  open_trade_date = 0
  holding_days = 0

  pnl = 0
  pnl_list = []
    
  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_rsi_14      = df.loc[i, 'RSI_14']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_MACD_hist   = df.loc[i, 'MACD_hist']
    now_bband_u     = df.loc[i, 'BBand_U']
    now_bband_m     = df.loc[i, 'BBand_M']
    now_bband_l     = df.loc[i, 'BBand_L']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']
    now_win_count   = df.loc[i, 'Win Count']

    # opening a position
    if (pos_opened == False and (now_open < now_bband_l and now_MACD > now_MACD_signal and now_MACD_hist > 0)):
      pos_opened = True
      open_trade_date = now_date
      open_price = now_open
      lot_size = pos_size/open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      df.at[i, 'Position'] = 1
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = initial_equity_value
      bb_macd_tradelist.append([now_date, df_name, pos_opened, open_price, lot_size, initial_equity_value])

    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif (pos_opened == True and ((now_MACD < now_MACD_signal) or \
                                  (lot_size * now_open > initial_equity_value * 1.2) or (lot_size * now_open < initial_equity_value * 0.95) or \
                                  (now_date == df.loc[(len(df)-1), 'Date']))):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      df.at[i, 'Position'] = 2
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = (close_price * lot_size)
      if (now_open * lot_size) < initial_equity_value:
        df.loc[i, 'MDD_dollar']   = -(initial_equity_value - (now_open * lot_size))
      else:
        df.loc[i, 'MDD_dollar']   = 0
      holding_days = now_date - open_trade_date
      df.at[i, 'Holding Period'] = holding_days.days
      
      pnl = round(((close_price * lot_size) - sell_commission) - initial_equity_value,2)
      df.at[i, 'PNL'] = pnl.round(2)
      if pnl > 0:
        df.at[i, 'Win Count'] = 1
        win_counter += 1
      pnl_list.append(pnl)
      bb_macd_tradelist.append([now_date, df_name, pos_opened, close_price, lot_size, (close_price*lot_size), pnl, holding_days])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      lot_size = 0
      open_trade_date = 0

    else:
      df.loc[i, 'Equity Value']   = (now_open * lot_size)
      df.loc[i, 'Lot Size']       = lot_size
      if (now_open * lot_size) < initial_equity_value:
        df.loc[i, 'MDD_dollar']   = initial_equity_value - (now_open * lot_size)
      else:
        df.loc[i, 'MDD_dollar']   = 0

  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].min().round(2)
  avg_hold_period = df['Holding Period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

In [13]:
print('BB_MACD Strategy: asset, no trade, total PNL, Avg PNL, Max MDD, Avg holding period, win rate')
print(backtest_bbmacd(BTCUSD))
print(backtest_bbmacd(ETHUSD))
print(backtest_bbmacd(BNBUSD))
print(backtest_bbmacd(SOLUSD))

BB_MACD Strategy: asset, no trade, total PNL, Avg PNL, Max MDD, Avg holding period, win rate
('BTCUSD', 7, 14.35, 2.05, -3.8, 0.0, 57.14)
('ETHUSD', 7, 101.6, 14.51, -11.66, 0.14, 57.14)
('BNBUSD', 10, 124.66, 12.47, -8.86, 0.1, 70.0)
('SOLUSD', 3, -7.87, -2.62, -9.31, 0.0, 33.33)


**2) MACD EMA Strategy**

*   Entry Rule: Open Price < 200 EMA, MACD > MACD signal and MACD > 0
*   Exit Rule: Hit profit target/ stop loss/ last day of trade, MACD < signal line




In [14]:
# defining the global list for storing trade list output
global macdema_tradelist
macdema_tradelist = []

pos_size = round(initial_capital/len(ticker_list),2)

In [15]:
# defining a backtest function based on trading rules, capture all transactions

def backtest_macdema(df):
  df_name = get_df_name(df)

  pos_opened = False
  open_price  = 0
  close_price = 0
  lot_size = 0

  initial_equity_value = 0
  win_counter = 0
  open_trade_date = 0
  holding_days = 0

  pnl = 0
  pnl_list = []
    
  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_200EMA      = df.loc[i, '200EMA']
    now_rsi_14      = df.loc[i, 'RSI_14']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_MACD_hist   = df.loc[i, 'MACD_hist']
    now_bband_u     = df.loc[i, 'BBand_U']
    now_bband_m     = df.loc[i, 'BBand_M']
    now_bband_l     = df.loc[i, 'BBand_L']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']
    now_win_count   = df.loc[i, 'Win Count']

    # opening a position 
    if (pos_opened == False and (now_open < now_200EMA and now_MACD > now_MACD_signal and now_MACD_hist > 0)):
      pos_opened = True
      open_trade_date = now_date
      open_price = now_open
      lot_size = pos_size/open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      df.at[i, 'Position'] = 1
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = initial_equity_value
      macdema_tradelist.append([now_date, df_name, pos_opened, open_price, lot_size, initial_equity_value])

    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif (pos_opened == True and ((now_MACD < now_MACD_signal) or \
                                  (lot_size * now_open > initial_equity_value * 1.2) or (lot_size * now_open < initial_equity_value * 0.95) or \
                                  (now_date == df.loc[(len(df)-1), 'Date']))):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      df.at[i, 'Position'] = 2
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = (close_price * lot_size)
      if (now_open * lot_size) < initial_equity_value:
        df.loc[i, 'MDD_dollar']   = -(initial_equity_value - (now_open * lot_size))
      else:
        df.loc[i, 'MDD_dollar']   = 0
      holding_days = now_date - open_trade_date
      df.at[i, 'Holding Period'] = holding_days.days
      
      pnl = round(((close_price * lot_size) - sell_commission) - initial_equity_value,2)
      df.at[i, 'PNL'] = pnl.round(2)
      if pnl > 0:
        df.at[i, 'Win Count'] = 1
        win_counter += 1
      pnl_list.append(pnl)
      macdema_tradelist.append([now_date, df_name, pos_opened, close_price, lot_size, (close_price*lot_size), pnl, holding_days])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      lot_size = 0
      open_trade_date = 0

    else:
      df.loc[i, 'Equity Value']   = (now_open * lot_size)
      df.loc[i, 'Lot Size']       = lot_size
      if (now_open * lot_size) < initial_equity_value:
        df.loc[i, 'MDD_dollar']   = initial_equity_value - (now_open * lot_size)
      else:
        df.loc[i, 'MDD_dollar']   = 0

  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].min().round(2)
  avg_hold_period = df['Holding Period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

In [16]:
print('MACD_EMA Strategy: asset, no trade, total PNL, Avg PNL, Max MDD, Avg holding period, win rate')
print(backtest_macdema(BTCUSD))
print(backtest_macdema(ETHUSD))
print(backtest_macdema(BNBUSD))
print(backtest_macdema(SOLUSD))

MACD_EMA Strategy: asset, no trade, total PNL, Avg PNL, Max MDD, Avg holding period, win rate
('BTCUSD', 228, 3176.44, 13.93, -67.64, 0.26, 62.72)
('ETHUSD', 220, 4394.0, 19.97, -127.99, 0.28, 63.64)
('BNBUSD', 218, 3579.76, 16.42, -116.56, 0.26, 68.81)
('SOLUSD', 244, 5593.11, 22.92, -105.42, 0.2, 58.2)


In [17]:
# create a for loop, put together trade summary list
macdema_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_macdema(df)
    macdema_result_list.append(test_result)

    macdema_result_df = pd.DataFrame(macdema_result_list)
    macdema_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = macdema_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    macdema_result_df = pd.DataFrame(macdema_result_list)
    macdema_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']

In [18]:
# PNL for MACD + 200EMA Strategy

#measure_date = df_list[1].loc[1, 'Date']
#end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: MACD + 200EMA')
#print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((macdema_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = macdema_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (macdema_result_df['No of Trade'].sum()))
print('Total PNL: USD', macdema_result_df['Total PNL'].sum().round(2))
print('Capital Deployed: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(macdema_result_df[macdema_result_df['No of Trade'] > 0]['Win Rate %'].mean(),2), '%')
macdema_result_df[macdema_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)


Trading Strategy: MACD + 200EMA
Total No of Trades Executed: 910
Total PNL: USD 16743.31
Capital Deployed: USD 5000
ROI:  334.87 %
Avg Win Rate:  63.34 %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %
3,SOLUSD,244,5593.11,22.92,-105.42,0.20,58.20
1,ETHUSD,220,4394.00,19.97,-127.99,0.28,63.64
2,BNBUSD,218,3579.76,16.42,-116.56,0.26,68.81
0,BTCUSD,228,3176.44,13.93,-67.64,0.26,62.72


In [23]:
all_trades = pd.DataFrame(macdema_tradelist).round(4)
all_trades = all_trades.drop_duplicates()
all_trades.columns = ['Date', 'Crypto', 'Open Trade', 'Open/ Close Price', 'Lot Size', 'Equity Value', 'PNL', 'Holding Period']

In [26]:
all_trades.head()

,Date,Crypto,Open Trade,Open/ Close Price,Lot Size,Equity Value,PNL,Holding Period
0,2022-01-01 07:00:00,BTCUSD,True,47034.8789,0.0266,1250.0000,NaN,NaT
1,2022-01-02 04:00:00,BTCUSD,False,47389.4141,0.0266,1259.4221,9.39,0 days 21:00:00
2,2022-01-02 16:00:00,BTCUSD,True,47333.5977,0.0264,1250.0000,NaN,NaT
3,2022-01-02 18:00:00,BTCUSD,False,47471.4609,0.0264,1253.6407,3.61,0 days 02:00:00
4,2022-01-03 10:00:00,BTCUSD,True,46967.1133,0.0266,1250.0000,NaN,NaT


**4) Graphing Price Outputs**

In [33]:
import plotly.graph_objects as go

In [43]:
fig = go.Figure(data=[go.Candlestick(x=BTCUSD['Date'], open=BTCUSD['Open'], high=BTCUSD['High'], low=BTCUSD['Low'], close=BTCUSD['Close'])])
fig.update_layout(title='BTC/USD Candlestick Price Chart - FY2022', yaxis_title='USD', xaxis_title='Date')
fig.show()